In [ ]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score,accuracy_score,get_scorer_names,roc_auc_score
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def stats_by_feat(train,test,feat_to_stat,feat_to_group,func):
    for df in (train,test):
        df[feat_to_stat+feat_to_group+func] = df[feat_to_group].map(df.groupby(feat_to_group)[feat_to_stat].aggregate(func))

In [ ]:
pd.options.display.max_columns=69

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
registration = pd.read_csv('registration.csv')

In [ ]:
registration

In [ ]:
test['ID'] = range(1,len(test)+1)

In [ ]:
train = pd.merge(train,registration,on='Student ID')
test = pd.merge(test,registration,on='Student ID')

In [ ]:
train

In [ ]:
registration

In [ ]:
# test = test.merge(registration,on='Student ID')

In [ ]:
# train = train.merge(registration,on='Student ID')

In [ ]:
train['Home Region'].value_counts()

In [ ]:
# for df in (train,test):
#     df['Gender']=df['Gender'].map({'أنثى':0,'ذكر':1})

In [ ]:
cat_cols = list(train.select_dtypes(exclude='number').columns)
for col in cat_cols:
    print(col,train[col].nunique())

In [ ]:
# traintest = pd.concat([train,test])

# tfidf = TfidfVectorizer()
# vec = tfidf.fit(traintest['Education Speaciality'].astype('str'))
# train_vectorized = tfidf.transform(train['Education Speaciality'].astype('str')).toarray()
# test_vectorized = tfidf.transform(test['Education Speaciality'].astype('str')).toarray()


In [ ]:
# train_vec = pd.DataFrame(train_vectorized,columns=tfidf.get_feature_names_out())
# test_vec = pd.DataFrame(test_vectorized,columns=tfidf.get_feature_names_out())

In [ ]:
# train = pd.concat([train,train_vec],axis=1)
# test = pd.concat([test,test_vec],axis=1)

In [ ]:
test

In [ ]:
train

# Feature Engineering

In [ ]:
for df in train,test:
    df['Program Start Date'] = pd.to_datetime(df['Program Start Date'])
    df['starting month'] = df['Program Start Date'].dt.month
    df['starting year'] = df['Program Start Date'].dt.year
    
    
    

In [ ]:
stats_by_feat(train,test,'Total Regestration','starting month','mean')


stats_by_feat(train,test,'PCRF','starting month','sum')
stats_by_feat(train,test,'PCRF','starting month','mean')
stats_by_feat(train,test,'PCRF','starting month','median')




# stats_by_feat(train,test,'N/A','starting month','mean')


# stats_by_feat(train,test,'Total Regestration','starting month','median')
# stats_by_feat(train,test,'Total Regestration','Home Region','mean')

In [ ]:
train

In [ ]:
for df in (train,test):
    df.drop(columns=['Program Start Date','Program End Date','Education Speaciality','Home City'],inplace=True)             

In [ ]:
train

In [ ]:
for df in train,test:
#     df['Age'] = (df['Age']/10).round().astype('str')
    df['presntation_place'] = df['Program Presentation Method'] + df['Home Region']
#     df['Program Days'] = pd.cut(df['Program Days'],bins=[0,7,30,90,120,150,200,250,300],labels=['tiny','not short','short','medium','not long','long','very long','super long'])
    # df['studen_program'] = df['Student ID']+df['Program ID']


# EDA

In [ ]:
train.describe()

In [ ]:
train['Y'].value_counts()

In [ ]:
import seaborn 

In [ ]:
seaborn.barplot(train,x=train['Program Days'],y=train['Y'])

In [ ]:
cat_cols = list(train.select_dtypes(exclude='number').columns)

for column in cat_cols:
    for df in train,test:
        df[column] = df[column].astype('str')

# for column in cat_cols:
#         traintest = pd.concat([train,test])
#         le.fit(traintest[column])
#         train[column] = le.transform(train[column])
#         test[column] = le.transform(test[column])

In [ ]:
THRESHOLD = 0.3

# Cross Validation

In [ ]:
model = CatBoostClassifier(random_state=42,cat_features=cat_cols)
stratfiedkfold = StratifiedKFold(shuffle=True,random_state=69,n_splits=10)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def validate(trainset,testset):
    model.fit(trainset.drop(columns='Y'),trainset['Y'])
    model.set_probability_threshold(THRESHOLD)
    pred = model.predict(np.array(testset.drop(columns='Y')))
    score = f1_score(pred,testset['Y'])
    return score

In [ ]:
train

In [ ]:
f1 = []
rocauc = []
for v_train,v_test in stratfiedkfold.split(train.drop(columns='Y'),train['Y'].astype('str')+train['Home Region']):
    train_v, test_V= train.iloc[v_train],train.iloc[v_test]

    # training,testing = mean_encoding(train_v,test_V)
    # train_v,test_V = mean_encoding(train_v,test_V,'Level of Education')
    f1.append(validate(train_v,test_V))



In [ ]:
np.array(f1).mean()


In [ ]:
np.array(f1).std()


In [ ]:
f1


In [ ]:
train

In [ ]:
model.fit(train.drop(columns='Y'),train['Y'])
model.set_probability_threshold(THRESHOLD)

In [ ]:
test

In [ ]:
seaborn.barplot(x=model.feature_importances_,y=model.feature_names_)

In [ ]:
pred = model.predict(test.drop(columns='ID'))

In [ ]:
sub = pd.DataFrame({
    'ID':test['ID'],
    'Y':pred
})

In [ ]:
# sub['Y'] = sub['Y'].map({1:0,0:1})

In [ ]:
sub.to_csv('submission.csv',index=False)

In [ ]:
seaborn.displot(train['Age'])

In [ ]:
seaborn.displot(test['Age'])


In [ ]:
seaborn.histplot(y=train['Home Region'])

In [ ]:
seaborn.histplot(y=test['Home Region'])


In [ ]:
seaborn.displot(train_v['Age'])


In [ ]:
seaborn.displot(test_V['Age'])